# SNIPPETS 

In [ ]:
# Afficher plusieurs  images dans une figure 

import matplotlib.pyplot as plt
import numpy as np

# Exemple de données : 4 matrices 100x100
img1 = np.random.rand(100, 100)
img2 = np.random.rand(100, 100)
img3 = np.random.rand(100, 100)
img4 = np.random.rand(100, 100)

# Création d'une figure avec 4 sous-figures (axes)
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Affichage des images
im1 = axes[0, 0].imshow(img1, cmap="gray")
axes[0, 0].set_title("Image 1")
fig.colorbar(im1, ax=axes[0, 0])

im2 = axes[0, 1].imshow(img2, cmap="gray")
axes[0, 1].set_title("Image 2")
fig.colorbar(im2, ax=axes[0, 1])

im3 = axes[1, 0].imshow(img3, cmap="gray")
axes[1, 0].set_title("Image 3")
fig.colorbar(im3, ax=axes[1, 0])

im4 = axes[1, 1].imshow(img4, cmap="gray")
axes[1, 1].set_title("Image 4")
fig.colorbar(im4, ax=axes[1, 1])

# Ajuster les espacements
plt.tight_layout()

plt.show()


# Nan-safe instructions  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- 1) Nettoyage NoData → NaN ----------------------------------------------
def to_nan(arr, nodata_values=(-9999, -32768, -3.4028235e38)):
    """
    Remplace les valeurs NoData par NaN.
    nodata_values : tuple de valeurs à traiter comme NoData.
    Retourne un tableau float (pour pouvoir porter des NaN).
    """
    out = arr.astype('float32', copy=True)
    for nd in nodata_values:
        np.putmask(out, out == nd, np.nan) # np.putmask(array, mask, value) remplace array[mask] par la value
    return out

# --- 2) Statistiques robustes (ignorent les NaN) -----------------------------
def nan_stats(arr):
    """
    Renvoie un dict avec stats nan-safe.
    """
    return {
        "min": float(np.nanmin(arr)),
        "q01": float(np.nanpercentile(arr, 1)),
        "q05": float(np.nanpercentile(arr, 5)),
        "median": float(np.nanmedian(arr)),
        "mean": float(np.nanmean(arr)),
        "std": float(np.nanstd(arr)),
        "q95": float(np.nanpercentile(arr, 95)),
        "q99": float(np.nanpercentile(arr, 99)),
        "max": float(np.nanmax(arr)),
        "count_valid": int(np.isfinite(arr).sum()),
        "count_nan": int(np.isnan(arr).sum())
    }

# --- 3) Affichage robuste (stretch percentile + colorbar) --------------------
def show_image(arr, title="Image", cmap="gray", p_lo=1, p_hi=99):
    """
    Affiche une image 2D avec colormap et colorbar.
    Etale la dynamique entre les percentiles p_lo et p_hi (nan-safe).
    """
    vmin = np.nanpercentile(arr, p_lo)
    vmax = np.nanpercentile(arr, p_hi)
    fig, ax = plt.subplots(figsize=(7, 5))
    img = ax.imshow(arr, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(title)
    cb = fig.colorbar(img, ax=ax, label="Valeurs de pixel")
    plt.tight_layout()
    plt.show()

# Remplissage optionnel (à utiliser avec prudence) ---------------------
# Si tu dois FORCÉMENT remplacer les NaN par une valeur (p.ex. 0) avant un calcul :
# band1_filled = np.nan_to_num(band1_clean, nan=0.0)
# Attention : ce choix peut biaiser des moyennes/ratios


# --- 4) Exemple d’utilisation ------------------------------------------------
# Supposons que `band1` soit ton array d’origine (np.ndarray) lu depuis rasterio.read(1)
# band1 = src.read(1)  # à titre d’exemple

# 4.1 Nettoyage NoData -> NaN
# band1_clean = to_nan(band1, nodata_values=(-9999, -32768))

# 4.2 Stats nan-safe
# stats = nan_stats(band1_clean)
# print(stats)

# 4.3 Percentiles spécifiques (nan-safe)
# p1, p99 = np.nanpercentile(band1_clean, (1, 99))
# print("1er percentile:", p1, "99e percentile:", p99)

# 4.4 Affichage avec stretch percentile et colorbar
# show_image(band1_clean, title="Bande 1 (NoData -> NaN)", cmap="gray", p_lo=1, p_hi=99)




# Difference raster between list of raster tuples (pre, post)

In [ ]:
import numpy as np
import rasterio

def compute_sar_temporal_differences(list_raster_pairs: list[tuple[str, str]]) -> list[tuple[dict, np.ndarray]]:
    """
    Computes the difference between pre and post image pairs for all bands, for all (pre, post) tuples 
    Includes a metadata consistency check between pre and post images.
    
    Args:
        list_raster_pairs (list of tuples): List containing (pre_path, post_path) strings.
        
    Returns:
        list of tuples: A list of (profile, diff_stack) for each pair.
    """

    diff_images = []

    for i, (path_pre, path_post) in enumerate(list_raster_pairs):
        with rasterio.open(path_pre) as src_pre, rasterio.open(path_post) as src_post:
            
            # 1. Shape, crs and S profile check 
            is_match = (src_pre.shape == src_post.shape and 
                        src_pre.crs == src_post.crs and 
                        np.allclose(np.array(src_pre.transform), np.array(src_post.transform), atol=1e-8))
            # Check if both rasters are perfectly aligned spatially by comparing their Affine transforms.
            # We use np.allclose with a small tolerance (1e-8) instead of strict equality (==) 
            # to account for potential floating-point rounding errors during geoprocessing.
            
            if not is_match:
                print(f"[SKIP] Pair {i+1}: Geometric mismatch between pre and post rasters.")
                continue

            # 2. Compute absolute difference
            # S1 imagery: we typically use the absolute difference of backscatter/textures
            diff_img = src_pre.read().astype(np.float32) - src_post.read().astype(np.float32)
            
            diff_images.append(diff_img)
            print(f"[INFO] Pair {i+1} processed successfully.")
            
    return diff_images



# Usage example
# list_raster_pairs = [("pre_1.tif", "post_1.tif"), ("pre_2.tif", "post_2.tif")]
# diffs = compute_raster_differences(list_raster_pairs)

# Mahotas functions  

In [ ]:
########## MAHOTAS ##########

import rasterio
import numpy as np
import mahotas as mh  # mahotas not present in vigisar environment, add it manually with conda install mahotas
from joblib import Parallel, delayed


def _compute_all_haralick_for_band(band_data: np.ndarray, window_size: int) -> np.ndarray:
    """
    Computes 13 Haralick textures AND keeps the original band data.
    Returns a stack of 14 layers (1 original + 13 textures).
    """
    h, w = band_data.shape
    pad = window_size // 2
    # Pre-allocate 14 layers: layer 0 is original, layers 1-13 are textures
    final_stack = np.zeros((14, h, w), dtype='float32')
    final_stack[0] = band_data # Keeping the original band

    # 1. Handle NaNs and identify valid data (mask pixel = True if NaN)
    mask = np.isnan(band_data)
    
    # 2. Robust Min-Max Scaling to 0-255
    # We calculate min/max only on valid pixels to avoid NaN interference
    if np.any(~mask):  # np.any(~mask) = True if at least one pixel not NaN
        b_min = np.nanmin(band_data)
        b_max = np.nanmax(band_data)
        
        # Avoid division by zero if the image is constant
        if b_max > b_min:
            # Linear stretch: (x - min) / (max - min) * 255
            clean_band = np.nan_to_num(band_data, nan=b_min)
            band_uint8 = ((clean_band - b_min) / (b_max - b_min) * 255).astype(np.uint8)
        else:
            band_uint8 = np.zeros((h, w), dtype=np.uint8)
    else:  # if no valid pixel, all the textures are just NaN images 
        return np.full((14, h, w), np.nan, dtype='float32') # 14 because original band is NaN as well

    # 3. Padding and Sliding Window
    padded = np.pad(band_uint8, pad, mode='reflect')  # adding pixels on the edges for the sliding window

    for i in range(h):
        for j in range(w):
            if mask[i, j]:
                final_stack[1:, i, j] = np.nan # if pixel not valid, NaN on all textures
                continue
            
            window = padded[i : i + window_size, j : j + window_size] # small window around the original image (i,j) pixel in the padded image
            try:
            # Compute the 13 Haralick features for the current window.
            # mahotas returns a (4, 13) matrix corresponding to 4 directions (0°, 45°, 90°, 135°).
            # We take the mean across axis 0 to obtain rotation-invariant descriptors.
            # This ensures forest textures remain consistent regardless of the sensor's/trees' orientation.
                features = mh.features.haralick(window).mean(axis=0)
                final_stack[1:, i, j] = features

            except:
             # Mathematical edge cases (e.g., a window with constant values) can cause Haralick failures.
            # Specifically, Correlation might be undefined if the standard deviation is zero.
            # We default to 0 to prevent the entire processing pipeline from crashing 
            # We can't ask mahotas to calculate everything but correlation, it's made to calculate everything at once
                final_stack[1:, i, j] = 0
                
    return final_stack


def generate_full_haralick_stack(
    input_stack: np.ndarray, 
    profile: dict, 
    window_size: int = 7
) -> tuple[dict, np.ndarray, list[str]]:
    """
    Generates a massive stack containing the original band 
    plus its 13 textures for every input band.
    """
    n_bands = input_stack.shape[0]
    haralick_names = [
        "ASM", "Contrast", "Correlation", "Variance", "IDM", 
        "SumAvg", "SumVar", "SumEnt", "Entropy", "DiffVar", 
        "DiffEnt", "IMC1", "IMC2"
    ]
    
    print(f"Generating 14 layers (1 Original + 13 Textures) for each of the {n_bands} bands...")

    # Parallelize the computation across all available CPU cores (n_jobs=-1)
    # to drastically reduce processing time for large S1 scenes
    # Each band is processed independently to generate its own 13 Haralick textures
    results = Parallel(n_jobs=-1)(
        delayed(_compute_all_haralick_for_band)(input_stack[b], window_size) 
        for b in range(n_bands)
    )
    
    # Concatenate all groups: Result shape (126, H, W)
    full_output = np.concatenate(results, axis=0)
    
    # Generate labels: Band_Original, Band_ASM, Band_Contrast...
    labels = []
    original_labels = ["G0_VH", "G0_VV", "q", "RVI", "DPSVI", "RFDI", "C_VH", "C_VV", "Cq"]
    for b_label in original_labels:
        labels.append(b_label) # Add original band name
        for h_label in haralick_names:
            labels.append(f"{b_label}_{h_label}") # Add texture names
            
    # Update profile count to 126 bands
    new_profile = profile.copy()
    new_profile.update({
        'count': full_output.shape[0],
        'dtype': 'float32',
        'nodata': np.nan
    })
    
    return new_profile, full_output, labels